In [3]:
import re
import time
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

### 곡 정보에서 가사 가져오기

In [13]:
def clean_song_name(song_name):
    song_name = re.sub('\\t', '', song_name)
    # song_name : ['곡명','TOMBOY']
    song_name = song_name.split()[1]
    return song_name

In [14]:
# 텍스트에 있는 \n과 \t를 제거하고 문장 양옆의 띄어쓰기를 지운다.
def clean_text(text):
    text = re.sub('\\t', '', text)
    text = re.sub('\\n', '', text)
    text = text.strip()
    return text

In [42]:
# {'songId':'가사'} 형식으로 dict를 만드는 함수
def make_lyrics_dict(songIdList):
    result_list = {}
    headers ={'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}
    
    for songId in songIdList:
        url = f"https://www.melon.com/song/detail.htm?songId={songId}"    
        res = requests.get(url, headers = headers)
        
        # 성공일 경우
        if res.status_code == 200:
            text = res.text
        
            bs = BeautifulSoup(res.text, 'html.parser')
            # lyrics 가사
            # song_name 가수명
            # artist 아티스트명
            lyrics = bs.select_one("#d_video_summary")
            
            # lyrics가 없을 경우 continue 
            if lyrics is None:
                continue

            lyrics = bs.select_one("#d_video_summary").text
            lyrics = clean_text(lyrics)


            '''        
            info = bs.select(".info")[1]
            song_name = info.select_one(".song_name").text
            song_name = clean_song_name(song_name)
            
            artist = info.select_one(".artist").text
            artist = clean_text(artist)
            '''
            result_list[songId] = lyrics

        time.sleep(0.3)
        
    return result_list

### 가사정보 csv 파일로 저장하기

In [5]:
# csv 파일 읽기
df = pd.read_csv('순위.csv', encoding='utf-8')

In [24]:
songIdList = list(df['song_id'])
print('org_len:', len(songIdList))
songIdList = list(set(songIdList))
print('set_len:', len(songIdList))

org_len: 1300
set_len: 1123


In [44]:
lyrics_dict = make_lyrics_dict(songIdList)

In [45]:
# CSV 파일 경로
csv_filename = "가사.csv"

# CSV 파일로 데이터 저장
with open(csv_filename, mode='w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['Song_ID', 'Lyrics']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    # CSV 파일의 헤더 작성
    writer.writeheader()

    # 딕셔너리 데이터를 CSV 파일에 작성
    for song_id, lyrics in lyrics_dict.items():
        writer.writerow({'Song_ID': song_id, 'Lyrics': lyrics})


### 앨범 정보에서 장르, 발매사, 기획사, 발매일 가져오기

In [8]:
# 장르: genre
# 발매일: release_date
# 발매사: publisher
# 기획사: agency

def make_album_info_dict(albumIdList):
    all_dict = {}
    headers ={'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}
    
    for albumId in albumIdList:
        url = f"https://www.melon.com/album/detail.htm?albumId={albumId}"
        res = requests.get(url, headers = headers)
        
        # 성공일 경우
        if res.status_code == 200:
            one_dict = {} 
            
            text = res.text
        
            bs = BeautifulSoup(res.text, 'html.parser')
            # lyrics 가사
            # song_name 가수명
            # artist 아티스트명
            geted_lst = bs.select_one(".list").text
            geted_lst= geted_lst.split()
        
            release_date = geted_lst[1]
            genre = geted_lst[3]
            publisher = geted_lst[5]
            agency = geted_lst[7]
        
            one_dict['release_date'] = release_date
            one_dict['genre'] = genre
            one_dict['publisher'] = publisher
            one_dict['agency'] = agency
            all_dict[albumId] = one_dict
        
        time.sleep(0.3)
    
    return all_dict

In [9]:
albumIdList = ['10909179', '10890384']
albumId = '10909179'

In [10]:
album_info_dict = make_album_info_dict(albumIdList)

In [11]:
album_info_dict

{'10909179': {'release_date': '2022.04.05',
  'genre': '댄스',
  'publisher': '카카오엔터테인먼트',
  'agency': '스타쉽엔터테인먼트'},
 '10890384': {'release_date': '2022.03.14',
  'genre': '록/메탈,',
  'publisher': '댄스,',
  'agency': '발매사'}}

In [12]:
album_info_dict['10909179']

{'release_date': '2022.04.05',
 'genre': '댄스',
 'publisher': '카카오엔터테인먼트',
 'agency': '스타쉽엔터테인먼트'}

In [13]:
album_info_dict['10890384']

{'release_date': '2022.03.14',
 'genre': '록/메탈,',
 'publisher': '댄스,',
 'agency': '발매사'}